# Get the data

In [3]:
!wget 'https://drive.google.com/uc?export=download&confirm=EfC2&id=0BwmD_VLjROrfTTljRDVZMFJnVWM'

/bin/sh: wget: command not found


In [2]:
!tar -xvzf cnn cnn.tgz

README.md                              songnames-holdout
Untitled.ipynb                         songnames-test
Untitled1.ipynb                        songnames-train
beam-search.ipynb                      tokenization.py
build-summary-data-set.ipynb           train-predicitons.txt
data.py                                train-summary-decoder-sparse.ipynb
model.py                               train-summary-dense.ipynb
model_decoder.py                       train-summary-transformer-sparse.ipynb
songnames


# Parse data and drop to disk

In [1]:
import glob

In [2]:
FILES = glob.glob('cnn/questions/training/*')

In [3]:
FILES

['cnn/questions/training/239c17682016c0c244cdf46200c5f740c8a2c9db.question',
 'cnn/questions/training/65c7b28e3ba0dacc5a85ab3ea1c9a635744bdf83.question',
 'cnn/questions/training/dc65fd491d2ac61c83037e5c1338cf6de19f95ee.question',
 'cnn/questions/training/603b08f191ac8bf06df441619dd912f446179065.question',
 'cnn/questions/training/0b3c640ebc44567eb3beb8b8297638d7090e8384.question',
 'cnn/questions/training/a3c5386bc005ef369b70267a5b4248b37052b64a.question',
 'cnn/questions/training/4dfd23aa3de5205cfe91b091b5365994105d29ba.question',
 'cnn/questions/training/36b0c8f1069460e5d997ad66c4ebff1c350c5000.question',
 'cnn/questions/training/18062cf32b6e91bbfa45e8645317499d417e2d87.question',
 'cnn/questions/training/b01d1b7f5bc30e17d727de4d8d85245b8ff2ffd0.question',
 'cnn/questions/training/530c4400a0a83900c2810e2795607af94a2447a5.question',
 'cnn/questions/training/9f17668a9cd8170fdad9788ed64e489e468cf9c3.question',
 'cnn/questions/training/3eb19c15e3b126ef702183f031e6bedfd8276ed9.question',

In [45]:
len(FILES)

380298

In [24]:
with open(FILES[0]) as f:
    content = f.read()

In [25]:
content.split('\n\n')

['http://web.archive.org/web/20140209002439id_/http://www.cnn.com:80/2013/11/25/showbiz/olivia-wise-death',
 '( @entity0 ) -- @entity1 , a teenager who refused to let an inoperable brain tumor kill her spirit , died monday . @entity1 gained fame in the last weeks of her 16 - year - long life when a @entity7 song she recorded in a @entity9 studio in september became a viral hit online . " she died peacefully in her home surrounded by the extraordinary love of her family , " a family statement sent to @entity0 said . the teenager said that she did n\'t want people crying at her funeral , but that they should celebrate her life , her mother wrote in a letter to @entity0 . her version of @entity23 \'s hit " @entity26 , " which she recorded in september after learning there were no more treatments available , drew the attention of @entity23 after it was published on @entity28 in october . " i was very moved and you sounded great , " @entity23 told her in a video posted on @entity28 . " i lo

In [26]:
class FORMAT:
    URL = 0
    CONTEXT = 1
    QUESTION = 2
    ANSWER = 3
    ENTITY_MAPPING = 4
    PLACEHOLDER_TOKEN = '@placeholder'

In [27]:
def parse(file):
    with open(file) as f:
        content = f.read()
        parts = content.split('\n\n')
        context = parts[FORMAT.CONTEXT]
        question = parts[FORMAT.QUESTION]
        answer = parts[FORMAT.ANSWER]
        summary = question.replace(FORMAT.PLACEHOLDER_TOKEN, answer)
        return '\t'.join([context, summary])

In [28]:
parsed_files = (parse(f) for f in FILES)

In [29]:
next(parsed_files)

'( @entity0 ) -- @entity1 , a teenager who refused to let an inoperable brain tumor kill her spirit , died monday . @entity1 gained fame in the last weeks of her 16 - year - long life when a @entity7 song she recorded in a @entity9 studio in september became a viral hit online . " she died peacefully in her home surrounded by the extraordinary love of her family , " a family statement sent to @entity0 said . the teenager said that she did n\'t want people crying at her funeral , but that they should celebrate her life , her mother wrote in a letter to @entity0 . her version of @entity23 \'s hit " @entity26 , " which she recorded in september after learning there were no more treatments available , drew the attention of @entity23 after it was published on @entity28 in october . " i was very moved and you sounded great , " @entity23 told her in a video posted on @entity28 . " i love you . a lot of people love you and that \'s why your video got to me . it moved everybody that saw it . " 

In [12]:
SUMMARY_DIRECTORY = 'cnn/summary'

In [31]:
import os
if not os.path.exists(SUMMARY_DIRECTORY):
    os.makedirs(SUMMARY_DIRECTORY)

In [34]:
import tqdm
for f in tqdm.tqdm(FILES):
    text = parse(f)
    dst_basename = os.path.basename(f)
    dst = os.path.join(SUMMARY_DIRECTORY, dst_basename)
    with open(dst, 'w') as f_out:
        f_out.write(text)

100%|██████████| 380298/380298 [07:35<00:00, 835.09it/s]


In [36]:
f_out

<_io.TextIOWrapper name='cnn/summary/1182fc9122f6dc29914383b2d0735ad478ad4534.question' mode='w' encoding='UTF-8'>

In [41]:
with open(dst) as f:
    print(f.read())

( @entity0 ) -- been thinking about wrestlers . no , not about @entity2 or @entity3 or @entity4 . i mean real wrestlers . wrestlers who wrestle for real . wrestlers who wo n't wrestle in the @entity11 if the @entity13 drop kicks their sport . it was revealed tuesday that the @entity13 is giving serious thought to the elimination of wrestling from @entity20 competition . been thinking about @entity21 . he was a deputy sheriff from @entity23 , @entity24 , when i watched him in the 1984 @entity26 on the night he became the first @entity27 ever to win a medal in @entity18 . it was shiny . it was gold . " knowing me , " he said , " i 'll probably have it bronzed . " been thinking about @entity34 . wrestling may be cut from @entity20 he was a high school and college wrestling champion from @entity38 , @entity39 , who was 29 when he finally made it to an @entity20 . he went to @entity41 in 2004 . he dedicated it to his older brother @entity43 , a former wrestler who died of heart failure righ

# Collect stats on data set to determine train parameters

In [13]:
import os
import tqdm
context_lengths = []
target_lengths = []
for f in tqdm.tqdm(FILES):
    dst_basename = os.path.basename(f)
    dst = os.path.join(SUMMARY_DIRECTORY, dst_basename)
    with open(dst, 'r') as f:
        context, target = f.read().split('\t')
    context_lengths.append(len(context.split(' ')))   
    target_lengths.append(len(target.split(' ')))

100%|██████████| 380298/380298 [02:20<00:00, 2706.24it/s]


In [14]:
import pandas as pd

In [17]:
df = pd.DataFrame(
    {'x': context_lengths,
     'y': target_lengths})

In [18]:
df.describe()

,x,y
count,380298.000000,380298.000000
mean,761.813517,12.470486
std,389.415990,3.214081
min,8.000000,1.000000
25%,452.000000,10.000000
50%,700.000000,12.000000
75%,1006.000000,14.000000
max,2000.000000,47.000000
